In [39]:
from pathlib import Path
import czifile
import numpy as np
import napari
import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.7
import pyclesperanto_prototype as cle
import skimage
from skimage import measure, exposure

In [40]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    images.append(str(file_path))
    
images

['raw_data\\well 1.czi',
 'raw_data\\well 10.czi',
 'raw_data\\well 11.czi',
 'raw_data\\well 12.czi',
 'raw_data\\well 13.czi',
 'raw_data\\well 14.czi',
 'raw_data\\well 15.czi',
 'raw_data\\well 2.czi',
 'raw_data\\well 3.czi',
 'raw_data\\well 4.czi',
 'raw_data\\well 5.czi',
 'raw_data\\well 6.czi',
 'raw_data\\well 7.czi',
 'raw_data\\well 8.czi',
 'raw_data\\well 9.czi']

In [41]:
image = images[12]

# Extract filename and well_id
file_path = Path(image)
filename = file_path.stem
well_id = filename.split(" ")[1]

# Read the image file and remove singleton dimensions
img = czifile.imread(image)
img = img.squeeze()

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = 4 # Use 2 or 4 for compression (None for lossless)

# Extract the stack containing the nuclei (2) and caspase channel (3)
nuclei_stack = img[2, :, ::slicing_factor, ::slicing_factor]
caspase_stack = img[3, :, ::slicing_factor, ::slicing_factor]

# Perform maximum intensity projections
nuclei_mip = np.max(nuclei_stack, axis = 0)
caspase_mip = np.max(caspase_stack, axis = 0)

In [42]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(nuclei_mip)
viewer.add_image(caspase_mip)

<Image layer 'caspase_mip' at 0x25166349310>

In [43]:
# Perform Gaussian Blur over nuclei_mip to get the outlines of the organoid shapes

gaussian_sigma_nuclei = 15

post_gaussian_img = skimage.filters.gaussian(nuclei_mip, sigma=gaussian_sigma_nuclei)
viewer.add_image(post_gaussian_img, name='Result of gaussian_blur')

# Apply Contrast Stretching to improve detection upon binarization
p2, p98 = np.percentile(post_gaussian_img, (2, 98))
img_rescale = exposure.rescale_intensity(post_gaussian_img, in_range=(p2, p98))
viewer.add_image(img_rescale, name='Result of contast stretching')


# Apply Otsu Thresholding to segment the organoid mask 

# organoid_mask = nsbatwm.threshold_otsu(img_rescale)
# viewer.add_labels(organoid_mask, name='Result of threshold_otsu')




<Image layer 'Result of contast stretching' at 0x2515ffef4f0>

In [44]:
organoid_mask = img_rescale > 0.2
viewer.add_labels(organoid_mask, name='Result of threshold')

<Labels layer 'Result of threshold' at 0x251621fd6a0>

In [45]:
post_mean_filter_caspase = None
post_mean_filter_caspase = cle.mean_box(caspase_mip, post_mean_filter_caspase, 2, 2, 0)
viewer.add_image(post_mean_filter_caspase, name='Result of box mean filter')
post_mean_filter_caspase = cle.pull(post_mean_filter_caspase)

caspase_mask = post_mean_filter_caspase > 15
viewer.add_labels(caspase_mask, name='Result of caspase threshold')

<Labels layer 'Result of caspase threshold' at 0x251651c9df0>

In [50]:
caspase_pos_area = (caspase_mask == 1) & (organoid_mask == 1)

In [51]:
viewer.add_labels(caspase_pos_area, name='Result of caspase + area')

<Labels layer 'Result of caspase + area' at 0x2516d954460>